In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
random_state = 42 
raw_dataset = pd.read_csv("./original_dataset/processed_data_encoded.csv") #data has X and Y, community 0-9
X = raw_dataset.drop(columns=["BMI", "DR"])
Y = pd.DataFrame(raw_dataset["DR"])
# Slice your data


X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)
df = pd.concat([X_FOR_FOLDS, Y_FOR_FOLDS], axis=1)
df.reset_index(drop=True, inplace=True)

In [2]:
import torch
import torch.nn as nn
class FeedForwardBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout=None, activation=nn.ReLU6):
        super().__init__()
        layers = [
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            activation()
        ]
        if dropout and dropout > 0:
            layers.append(nn.Dropout(dropout))
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x)
def print_activations(x, name):
    print(f"{name}: min={x.min().item()}, max={x.max().item()}")
    return x
class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout= None, hidden_dim2= None):
        super().__init__()
        
        # A couple of FeedForward blocks
        self.block1 = FeedForwardBlock(input_dim, hidden_dim, dropout= None)
        self.block2 = FeedForwardBlock(hidden_dim, hidden_dim2, dropout = dropout/2)
        self.block3 = FeedForwardBlock(hidden_dim2, output_dim, dropout)

        # Final output layer (could be softmax, sigmoid, or whatever your target is)
        self.output_layer = nn.Linear(output_dim, 1)  # Just in case you're doing regression or binary classification

    def forward(self, x):
        x = self.block1(x)
        # x = print_activations(x, "after block1")
        x = self.block2(x)
        # x = print_activations(x, "after block2")
        x = self.block3(x)
        # x = print_activations(x, "after block3")
        x = self.output_layer(x)  # Final linear layer
        return x

#* Test the model
test_model = MyModel(input_dim = 20,
                     hidden_dim = 64,
                     hidden_dim2 = 16,
                     output_dim = 4,
                     dropout = .2)
print(test_model)


MyModel(
  (block1): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
  )
  (block2): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=64, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
      (3): Dropout(p=0.1, inplace=False)
    )
  )
  (block3): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=16, out_features=4, bias=True)
      (1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (output_layer): Linear(in_features=4, out_features=1, bias=True)
)


In [3]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device, threshold = 0.5):
    # if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
    #     raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")
    best_val_loss = float('inf')
    best_model_state = None
    wait = 0
    criterion.to(device) #? Move criterion to device
    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            
            
            torch.set_printoptions(threshold=float('inf'))
            
            assert not torch.isnan(inputs).any(), "Input has NaNs"
            assert not torch.isinf(inputs).any(), "Input has Infs"
            outputs = model(inputs) #? Forward pass through the model
            assert not torch.isnan(outputs).any(), "Model output has NaNs"
            assert not torch.isinf(outputs).any(), "Model output has Infs"
            loss = criterion(outputs, labels) #? Calculate loss
            assert not torch.isnan(loss).any(), "Model loss has NaNs"
            loss.backward() #? Backpropagation
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            running_loss += loss.item()
            optimiser.step() #? Update weights
            scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                
                outputs = model(inputs)
                # labels = labels.cpu() 
                print(outputs.shape, labels.shape)
                
                
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
            avg_val_loss = val_loss / len(val_loader)
        loss_ratio = val_loss / train_loss    
        pos_weight = loss_ratio  # or any other function of loss_ratio you choose
    
        # Update criterion with new pos_weight
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            wait = 0
        elif avg_val_loss*0.95 <= best_val_loss:
                wait = 0
        else:
            wait += 1
        if wait >= patience:
            print(f"Early stopping triggered at epoch {epoch}, best val loss: {best_val_loss:.4f}")
            break
        print(f"Epoch: {epoch}".ljust(12), f"training loss:{train_loss:.3f}".ljust(16), f"best_val_loss:{best_val_loss:.3f}".ljust(12), f"Val Loss: {avg_val_loss:.3f}", f"Scheduler lr: {scheduler.get_last_lr()}".ljust(50),end="\r")
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                # predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                predictions = (torch.sigmoid(outputs) >= threshold).float().cpu().numpy()
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy=accuracy_score(labels, predictions) 
    precision=precision_score(labels, predictions, pos_label=1, zero_division=0)
    recall=recall_score(labels, predictions, pos_label=1)
    f1=f1_score(labels, predictions, pos_label=1)
    auc=roc_auc_score(labels, predictions)
    
    return model, accuracy, precision, recall, f1, auc

Using cuda


In [4]:
# from sklearn.preprocessing import RobustScaler
# kFolds = FOLDS_GENERATOR(df, n_splits=5, random_state=42,             
#                             OD_majority = IsolationForest(contamination=0.1, random_state=random_state), 
#                             OD_minority = IsolationForest(contamination=0.01, random_state=random_state), 
#                             oversampler_first = False,
#                             synthesizer =  "TVAE",
#                             epochs = 100,
#                             n_synthetic_data =1000,
#                             scaler=RobustScaler(),      
#                                )

In [5]:
from Training_Helper_Functions import *
from Preprocessing_Functions import * 
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler
import numpy as np
import optuna
from torch import optim
def maximise_combined_score(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    epochs = 10000
    random_state = 42
    
    # Scaler
    scaler = RobustScaler() # Or StandardScaler() - You can also make this a hyperparameter if you want

    # kFolds = FOLDS_GENERATOR(df, n_splits=5, random_state=42,             
    #                         OD_majority = IsolationForest(contamination=trial.suggest_float("contamination_majority", 0.01, 0.4), random_state=random_state), 
    #                         OD_minority = IsolationForest(contamination=trial.suggest_float("contamination_minority", 0.01, 0.2), random_state=random_state), 
    #                         oversampler_first = trial.suggest_categorical("oversampler_first", [False]),
    #                         synthesizer =  trial.suggest_categorical("synthesizer", ["TVAE", "CTGAN"]),
    #                         epochs = trial.suggest_int("epochs", 100, 100, ),
    #                         n_synthetic_data = trial.suggest_int("n_synthetic_data", 1000, 10000, step=1000),
    #                         scaler=scaler,      
    #                            ) # Pass outlier models and scaler
    kFolds = FOLDS_GENERATOR(
                            df,
                            n_splits=5,
                            random_state=42,             
                            OD_majority=IQRDetector(factor=trial.suggest_float("iqr_factor_majority", 1.0, 3.0)),
                            OD_minority=IQRDetector(factor=trial.suggest_float("iqr_factor_minority", 1.5, 3.0)),
                            oversampler_first=trial.suggest_categorical("oversampler_first", [True, False]),
                            synthesizer=trial.suggest_categorical("synthesizer", ["TVAE", "CTGAN"]),
                            epochs=trial.suggest_int("epochs", 100, 100, step=100),
                            n_synthetic_data=trial.suggest_int("n_synthetic_data", 1000, 1000, step=1000),
                            scaler=scaler
                        )
    # Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 512, 512, step=32)
    hidden_dim2 = trial.suggest_int("hidden_dim2", 256, 256, step=32)
    output_dim = trial.suggest_int("output_dim", 96, 96, step=2)

    dropout = trial.suggest_float("dropout", 0.0, 0.4)
    threshold = trial.suggest_float("threshold", 0.3, 0.8)
    # dropout = None
    initial_lr = trial.suggest_float("initial_lr", 1e-4, 1e-4, log=True)
    max_lr = trial.suggest_float("max_lr", 1e-4, 1e-4, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)


    # Loss function hyperparameters
    criterion_choice = "FocalLoss"
    # Hyperparameter exploration optimization
    if criterion_choice == "BCEWithLogitsLoss":
        pos_weight = trial.suggest_int("pos_weight", 1, 1)
        alpha = None
        gamma = None
    elif criterion_choice == "FocalLoss":
        pos_weight = None
        alpha = trial.suggest_float("alpha", 0.25, 0.75)
        gamma = trial.suggest_float("gamma", 1.0, 5.0)
    else:
        pos_weight = None

    # Initialize lists for metrics across folds
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    # Cross-validation loop
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        print(f"Fold {fold}:")
        # Create DataLoader for current fold
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64,
                                                            device=device)
        # Instantiate and initialize the model
        model = MyModel(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, hidden_dim2=hidden_dim2,
                        output_dim=output_dim, dropout=dropout)
        model.to(device)
        model.apply(init_weights)

        # Map the choice to the actual loss function
        criterion = criterion_mapping(criterion_choice, pos_weight, alpha, gamma)
        optimiser = optim.Adam(model.parameters(), lr=initial_lr, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.CyclicLR(
            optimiser,
            base_lr=1e-5,
            max_lr=max_lr,
            cycle_momentum=False)

        # Train and evaluate the model on the current fold
        model, accuracy, precision, recall, f1, auc = train_and_evaluate(
            model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=epochs, patience=100,
            device=device, threshold=threshold
        )
        print(f"Accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")
        del model
        del train_loader
        del val_loader

        # Append the metrics from the current fold
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)
        break

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    # combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5
    combined_score = avg_f1

    return combined_score

Using cuda


c:\Users\TAN LE ZHAN\Documents\GitHub\ADL2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import threading
import optuna
from optuna_dashboard import run_server

def start_dashboard():
    run_server(storage)

storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(direction="maximize", storage=storage, study_name="Basic")

# Start dashboard in a separate thread
dashboard_thread = threading.Thread(target=start_dashboard, daemon=True)
dashboard_thread.start()

# Run optimization
study.optimize(maximise_combined_score, n_trials=30)

# After optimization, print results
print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-20 19:02:47,397] A new study created in memory with name: Basic
Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

c:\Users\TAN LE ZHAN\Documents\GitHub\ADL2\.venv\Lib\site-packages\sdv\single_table\base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Using device: cuda
Original class distribution: DR
0.0    4129
1.0     464
Name: count, dtype: int64
After OD, majority: 3479
After OD, minority: 371
Before oversampling & synthetic data: DR 
0.0    3479
1.0     371
Name: count, dtype: int64

Applying SMOTENC oversampling...
               Age       Gender    Community         UAlb           Ucr  \
count  6958.000000  6958.000000  6958.000000  6958.000000   6958.000000   
mean     63.235204     0.533918     4.096579    19.931105   3832.922120   
std       7.125597     0.498884     3.072014    25.988688   5471.961292   
min      36.000000     0.000000     0.000000     0.100000      1.000000   
25%      59.000000     0.000000     1.000000     4.801043      6.000000   
50%      63.550490     1.000000     4.000000    10.295181     11.000000   
75%      68.000000     1.000000     7.000000    22.757483   7264.750000   
max      92.000000     1.000000     9.000000   174.800000  22572.000000   

              UACR           TC           TG    

Loss: 26.642: 100%|██████████| 100/100 [00:34<00:00,  2.93it/s]


After oversampling & synthetic data: DR 
0.0    3989
1.0    3969
Name: count, dtype: int64
Fold: 1, Train: (7958, 28), Test: (1149, 28)
Fold 1:
torch.Size([1149, 1]) torch.Size([1149, 1])
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.026 Val Loss: 0.026 Scheduler lr: [1.5579999999999986e-05]            
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.026 Val Loss: 0.495 Scheduler lr: [2.116000000000001e-05]             
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.026 Val Loss: 0.550 Scheduler lr: [2.6739999999999998e-05]            
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.026 Val Loss: 0.555 Scheduler lr: [3.232000000000002e-05]             
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.026 Val Loss: 0.591 Scheduler lr: [3.7900000000000006e-05]            
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.026 Val Loss: 0.669 Scheduler lr: [4.347999999999999e-05]             
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.026 Val Loss: 0.672 Scheduler lr: [4.906000

[I 2025-04-20 19:04:29,492] Trial 0 finished with value: 0.26976744186046514 and parameters: {'iqr_factor_majority': 2.8973683862076705, 'iqr_factor_minority': 2.4125576703146225, 'oversampler_first': True, 'synthesizer': 'TVAE', 'epochs': 100, 'n_synthetic_data': 1000, 'hidden_dim': 512, 'hidden_dim2': 256, 'output_dim': 96, 'dropout': 0.05825174422889084, 'threshold': 0.7093751484122822, 'initial_lr': 0.0001, 'max_lr': 0.0001, 'weight_decay': 5.127688485022286e-05, 'alpha': 0.27930148450962344, 'gamma': 3.7914534719814665}. Best is trial 0 with value: 0.26976744186046514.


...

torch.Size([1149, 1]) torch.Size([1149, 1])
Early stopping triggered at epoch 101, best val loss: 0.0372
Accuracy: 0.7093, precision: 0.1794, recall: 0.5259, f1: 0.2675, auc: 0.6279
Using device: cuda
Original class distribution: DR
0.0    4129
1.0     464
Name: count, dtype: int64
After OD, majority: 3288
After OD, minority: 331
Before oversampling & synthetic data: DR 
0.0    3288
1.0     331
Name: count, dtype: int64

Applying SMOTENC oversampling...
               Age       Gender    Community         UAlb           Ucr  \
count  6576.000000  6576.000000  6576.000000  6576.000000   6576.000000   
mean     63.500069     0.557482     3.957421    18.412445   3543.570089   
std       7.029458     0.496723     3.087472    23.468294   5219.720507   
min      37.000000     0.000000     0.000000     0.100000      1.000000   
25%      59.000000     0.000000     1.000000     4.800000      6.000000   
50%      64.000000     1.000000     4.000000     9.700000     11.000000   
75%      68.00000

c:\Users\TAN LE ZHAN\Documents\GitHub\ADL2\.venv\Lib\site-packages\sdv\single_table\base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Loss: 28.675: 100%|██████████| 100/100 [00:30<00:00,  3.25it/s]


After oversampling & synthetic data: DR 
0.0    3805
1.0    3771
Name: count, dtype: int64
Fold: 1, Train: (7576, 28), Test: (1149, 28)
Fold 1:
torch.Size([1149, 1]) torch.Size([1149, 1])
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.294 Val Loss: 0.294 Scheduler lr: [1.5310000000000014e-05]            
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.294 Val Loss: 1.066 Scheduler lr: [2.061999999999999e-05]             
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.294 Val Loss: 1.099 Scheduler lr: [2.5930000000000004e-05]            
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.294 Val Loss: 1.098 Scheduler lr: [3.123999999999998e-05]             
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.294 Val Loss: 1.141 Scheduler lr: [3.6549999999999994e-05]            
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.294 Val Loss: 1.163 Scheduler lr: [4.186000000000001e-05]             
torch.Size([1149, 1]) torch.Size([1149, 1])oss:0.294 Val Loss: 1.137 Scheduler lr: [4.716999

[I 2025-04-20 20:05:56,193] Trial 29 finished with value: 0.30198019801980197 and parameters: {'iqr_factor_majority': 2.2528404078809046, 'iqr_factor_minority': 1.771157021590618, 'oversampler_first': True, 'synthesizer': 'TVAE', 'epochs': 100, 'n_synthetic_data': 1000, 'hidden_dim': 512, 'hidden_dim2': 256, 'output_dim': 96, 'dropout': 0.2547176504360028, 'threshold': 0.7527691264110944, 'initial_lr': 0.0001, 'max_lr': 0.0001, 'weight_decay': 3.6014093403011885e-05, 'alpha': 0.25763645285935854, 'gamma': 3.6207963878884915}. Best is trial 21 with value: 0.3055555555555556.


torch.Size([1149, 1]) torch.Size([1149, 1])
Early stopping triggered at epoch 101, best val loss: 0.2938
Accuracy: 0.7546, precision: 0.2118, recall: 0.5259, f1: 0.3020, auc: 0.6531
Best trial:
  Combined score: 0.3055555555555556
  Best hyperparameters:
    iqr_factor_majority: 1.5764010798916201
    iqr_factor_minority: 1.8330470555241232
    oversampler_first: True
    synthesizer: TVAE
    epochs: 100
    n_synthetic_data: 1000
    hidden_dim: 512
    hidden_dim2: 256
    output_dim: 96
    dropout: 0.35945936018631525
    threshold: 0.5563855072726257
    initial_lr: 0.0001
    max_lr: 0.0001
    weight_decay: 9.34410803708391e-06
    alpha: 0.46413415191838836
    gamma: 4.8799880473043835


In [7]:
# import threading
# import optuna
# from optuna_dashboard import run_server
# # !fuser -k 8080/tcp

# # Define your persistent storage
# storage = "sqlite:///opt6.db"

# # Create or load your study
# study_name = "optuna6"
# try:
#     study = optuna.load_study(study_name=study_name, storage=storage)
# except KeyError:
#     study = optuna.create_study(study_name=study_name, direction="maximize", storage=storage)

# # Start Optuna Dashboard in a separate thread
# dashboard_thread = threading.Thread(target=lambda: run_server(storage), daemon=True)
# dashboard_thread.start()

# # Run optimization
# # Ensure the 'DR' column exists in the DataFrame
# if 'DR' not in df.columns:
#     raise KeyError("'DR' column is missing in the DataFrame. Please ensure it is present before running the optimization.")

# study.optimize(maximise_combined_score, n_trials=1000)

# # Print results
# print("Best trial:")
# trial = study.best_trial
# print(f"  Combined score: {trial.value}")
# print("  Best hyperparameters:")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")
